In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.amp import GradScaler, autocast # NEW: Use robust torch.amp
from torch.utils.data import Dataset, DataLoader

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import glob
from collections import defaultdict
from PIL import Image

import math
import shutil
import random
from typing import Callable, Optional, Dict, Any, Tuple, List

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler


/home/mohammed/miniconda3/envs/bioenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/mohammed/miniconda3/envs/bioenv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/mohammed/miniconda3/envs/bioenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from utils.config import CFG
def get_id(path):
    return os.path.splitext(os.path.basename(path))[0]

base_directory = CFG.DATA_DIR
train_csv = pd.read_csv(
    os.path.join(base_directory , "train.csv")
)
train_imgs_path = glob.glob(\
    os.path.join(base_directory , "train" , "*.jpg")
)

dict_ = defaultdict(list)

for path in train_imgs_path:
    id_ = get_id(path)
    cols_to_include = ["target_name" , "target" , "Sampling_Date" , "State" ,"Pre_GSHH_NDVI" ]
    img_info = train_csv[train_csv["sample_id"].str.split("_" , expand=True)[0]==id_][cols_to_include].copy()
    extra_info = img_info[[ "Sampling_Date" , "State" ,"Pre_GSHH_NDVI"]].iloc[0,:].to_dict()
    info_dict = img_info.set_index("target_name").to_dict()['target']
    info_dict["id"] = id_
    info_dict["image_path"] = path
    info_dict.update(extra_info)
    for k  , v in info_dict.items():
        dict_[k].append(v)

df = pd.DataFrame(dict_)

In [13]:
df

,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,id,image_path,Sampling_Date,State,Pre_GSHH_NDVI
0,0.4800,28.3224,9.1208,37.9232,9.6008,ID21377800,/home/mohammed/projects/bioimage/utils/../data...,2015/6/26,Tas,0.59
1,0.0000,1.3446,66.5554,67.9000,66.5554,ID1211362607,/home/mohammed/projects/bioimage/utils/../data...,2015/2/24,NSW,0.58
2,0.0000,16.3603,4.5809,20.9412,4.5809,ID1249094008,/home/mohammed/projects/bioimage/utils/../data...,2015/6/26,Vic,0.71
3,5.6182,5.6182,19.6636,30.9000,25.2818,ID141370843,/home/mohammed/projects/bioimage/utils/../data...,2015/8/14,Vic,0.72
4,2.7970,19.2502,11.3527,33.4000,14.1498,ID325799913,/home/mohammed/projects/bioimage/utils/../data...,2015/10/13,Vic,0.37
...,...,...,...,...,...,...,...,...,...,...
352,0.5231,24.8461,9.0231,34.3923,9.5462,ID196516535,/home/mohammed/projects/bioimage/utils/../data...,2015/5/18,Tas,0.45
353,0.0000,24.7869,17.2131,42.0000,17.2131,ID1268934251,/home/mohammed/projects/bioimage/utils/../data...,2015/10/13,Vic,0.42
354,5.3000,4.3000,6.9000,16.5000,12.2000,ID1451025862,/home/mohammed/projects/bioimage/utils/../data...,2015/11/10,Tas,0.53
355,7.4266,5.7388,7.8767,21.0421,15.3034,ID972274220,/home/mohammed/projects/bioimage/utils/../data...,2015/11/10,Tas,0.56


In [41]:
80/5

16.0

In [43]:
df.index

RangeIndex(start=0, stop=357, step=1)

In [63]:
pd.read_csv("kf_runs_resnet/per_fold_r2_table.csv")

,fold_1_Dry_Total_g,fold_2_Dry_Total_g,fold_3_Dry_Total_g,fold_4_Dry_Total_g,fold_1_GDM_g,fold_2_GDM_g,fold_3_GDM_g,fold_4_GDM_g,fold_1_Dry_Green_g,fold_2_Dry_Green_g,...,fold_3_Dry_Clover_g,fold_4_Dry_Clover_g,fold_1_Dry_Dead_g,fold_2_Dry_Dead_g,fold_3_Dry_Dead_g,fold_4_Dry_Dead_g,total_r2_fold_1,total_r2_fold_2,total_r2_fold_3,total_r2_fold_4
0,-3.599555,-8.643104,-0.098053,0.374384,-1.236259,-2.910416,-0.117231,0.329133,-0.256960,-2.336451,...,-69.183258,-1.043687,-4.277358,-2.544110,0.110704,0.0,-2.542273,-5.496000,-7.091452,0.007886
1,0.172598,-4.204689,-0.799680,NaN,-0.145163,-1.366648,-1.137133,NaN,-0.178268,-1.163113,...,-44.573711,NaN,-0.690994,-1.021876,0.031171,NaN,-0.067772,-2.627410,-5.313727,NaN
2,-0.033468,-3.618507,0.116001,NaN,-0.630065,-1.381255,-0.256987,NaN,-0.663392,-1.473852,...,-42.084881,NaN,-0.270017,-0.625669,-0.122303,NaN,-0.274815,-2.310601,-4.316464,NaN
3,-0.231966,-6.920375,0.247591,NaN,-0.935984,-3.102311,-0.105272,NaN,-1.051206,-3.548209,...,-67.584274,NaN,-0.133016,-0.930620,-0.201497,NaN,-0.448746,-4.559735,-6.945358,NaN
4,0.032064,-0.190954,-0.136395,NaN,-0.615181,0.273697,-0.868992,NaN,-0.831430,0.166373,...,-59.626259,NaN,-0.121689,-0.451662,0.080837,NaN,-0.234224,-0.070684,-6.624190,NaN
5,0.269198,-2.649766,-0.043021,NaN,-0.040079,-1.338453,-1.219093,NaN,-0.247123,-2.533659,...,-36.993530,NaN,-0.395469,-0.170508,-0.095166,NaN,0.045496,-1.866736,-4.240698,NaN
6,0.188621,-1.292012,-0.011298,NaN,0.019556,-0.374946,-0.520358,NaN,-0.151120,-1.495372,...,-54.313766,NaN,-0.455643,-0.400717,0.040068,NaN,0.016251,-0.911407,-5.890442,NaN
7,0.278032,0.206685,0.045978,NaN,-0.137623,0.341926,-0.996061,NaN,-0.387933,-0.135586,...,-81.999374,NaN,-0.129711,-0.255633,-0.120815,NaN,0.043836,0.135410,-9.451300,NaN
8,0.259003,-0.106812,0.328448,NaN,-0.170355,0.186273,-0.663858,NaN,-0.437374,-2.124055,...,-35.361271,NaN,-0.022565,-0.451415,-0.172525,NaN,0.058801,-0.319521,-3.827723,NaN
9,0.280282,0.233117,0.237608,NaN,-0.070399,0.422716,-1.457374,NaN,-0.326466,-0.073453,...,-48.322067,NaN,-0.069779,-0.244742,-0.356515,NaN,0.117916,0.174498,-5.955480,NaN


In [64]:
targets = CFG.ALL_TARGETS
n_splits = 5
save_every=5
n_epochs = 89 
start_fold , start_epoch , checkpoint = 0, 0 , None
df_cols = [f"fold_{fold}_{target}" for target in targets for fold in range(1 , n_splits+1)] + [f"total_r2_fold_{fold}" for fold in range(1 , n_splits+1)]
epoch_nums = range(save_every , n_epochs ,save_every)

epoch_labels = [f"epoch_{i}" for i in epoch_nums]
df_table = pd.DataFrame(columns=df_cols)
df_table




,fold_1_Dry_Total_g,fold_2_Dry_Total_g,fold_3_Dry_Total_g,fold_4_Dry_Total_g,fold_5_Dry_Total_g,fold_1_GDM_g,fold_2_GDM_g,fold_3_GDM_g,fold_4_GDM_g,fold_5_GDM_g,...,fold_1_Dry_Dead_g,fold_2_Dry_Dead_g,fold_3_Dry_Dead_g,fold_4_Dry_Dead_g,fold_5_Dry_Dead_g,total_r2_fold_1,total_r2_fold_2,total_r2_fold_3,total_r2_fold_4,total_r2_fold_5


In [ ]:


def model_constructor(constructor_params):
    if constructor_params is None:
        constructor_params = {}
    return BiomassModel(
        **constructor_params
        )


model_kwargs = dict(constructor_params = dict(
    model_name = "eva02_base_patch14_448.mim_in22k_ft_in1k",
    dropout_rate = 0.2,
    create_model_params = dict(
            pretrained=True,
            num_classes=0,
            global_pool='avg'
            )
)
)